In [1]:
import random
import pandas as pd
import numpy as np
import time

In [2]:
def diff(Rule1, Rule2):
    if Rule1[-1] != Rule2[-1]:
        return -1
    m = len(Rule1)
    count = 0
    for i in range(m-1):
        if Rule1[i] == Rule2[i]:
            count += 1
    return count/m

In [3]:
def sampling(base, ran, e, k = 2):
    

    num = len(base)
    R = []
    list_index = []
    while (len(R)<num*ran/100):
        index = random.randrange(num)
        while (index in list_index):
            index = random.randrange(num)
        # list_index.append(index)
        T = []
        T.append(index)
        for i in range(index-k,index+k+1):
            if i < num:
                if i in list_index:
                    continue
                else:
                    if diff(base[i],base[index]) < 1 - e:
                        T.append(i)
        for i in T:
            temp = 0
            for j in range(len(R)):
                if diff(base[i],R[j]) < 1 - e:
                    continue
                else:
                    temp = 1
            if temp:
                T.remove(i)
        for i in T:
            R.append(base[i])
            list_index.append(i)
    
    return R



In [4]:
def combination(k, n):
    if k == 0 or k == n:
        return 1
    if k == 1:
        return n
    return combination(k - 1, n - 1) + combination(k, n - 1)


def caculateA(base):
    colum = len(base[0])
    row = len(base)
    A = np.zeros((row, combination(4, colum - 1)))

    for r1 in range(row):
        k = [0] * combination(4, colum - 1)
        temp = 0
        for a in range(0, colum - 4):
            for b in range(a + 1, colum - 3):
                for c in range(b + 1, colum - 2):
                    for d in range(c + 1, colum - 1):
                        for r2 in range(row):
                            if base[r1][a] == base[r2][a] and base[r1][b] == base[r2][b] and base[r1][c] == base[r2][c] and base[r1][d] == base[r2][d]:
                                k[temp] += 1

                        A[r1][temp] = k[temp] / row
                        temp += 1
    print("done A")
    return A


def caculateM(base):
    colum = len(base[0])
    row = len(base)
    M = np.zeros((row, colum - 1))
    for t1 in range(row):
        k = [0] * (colum - 1)
        temp = 0
        for i in range(colum - 1):
            for t2 in range(row):
                if base[t1][i] == base[t2][i] and base[t1][colum - 1] == base[t2][colum - 1]:
                    k[temp] += 1
            M[t1][temp] = k[temp] / row
            temp += 1

    return M


def caculateB(base, A, M):
    colum = len(base[0])
    row = len(base)
    B = np.zeros((row, combination(3, colum - 1)))

    for r in range(row):
        temp = 0
        for a in range(0, colum - 3):
            for b in range(a + 1, colum - 2):
                for c in range(b + 1, colum - 1):
                    B[r][temp] = sum(A[r]) * min(M[r][a], M[r][b], M[r][c])
                    temp += 1
    print("done B")
    return B


def caculateC(base, B):
    colum = len(base[0])
    row = len(base)
    cols = 2 * combination(3, colum - 1)
    C = np.zeros((row, cols))

    for r1 in range(row):
        temp = 0
        for i in range(2):
            for a in range(0, (colum - 3)):
                for b in range(a + 1, (colum - 2)):
                    for c in range(b + 1, (colum - 1)):
                        for r2 in range(row):
                            if base[r1][a] == base[r2][a] and base[r1][b] == base[r2][b] and base[r1][c] == base[r2][c] and base[r2][colum - 1] == i:
                                C[r1][temp] += B[r2][temp % combination(3, colum - 1)]
                        # print(temp,":",temp//combination(3,colum-1))
                        temp += 1
    print("done C")
    return C

In [5]:
def FISA(base, C, list):
    colum = len(base[0])
    row = len(base)

    cols = combination(3, (colum - 1))
    C0 = [0] * cols
    C1 = [0] * cols

    t = 0
    for a in range(0, colum - 3):
        for b in range(a + 1, colum - 2):
            for c in range(b + 1, colum - 1):
                for r in range(row-1):
                    if base[r][a] == list[a] and base[r][b] == list[b] and base[r][c] == list[c] and base[r][colum-1] == 0:
                        C0[t] = C[r][t + 0 * cols]
                        # break
                    if base[r][a] == list[a] and base[r][b] == list[b] and base[r][c] == list[c] and base[r][colum-1] == 1:
                        C1[t] = C[r][t + 1 * cols]
                        # break
                t += 1
    #print(t)

    D0 = max(C0) + min(C0)
    D1 = max(C1) + min(C1)

    #print(D0, max(C0), min(C0))
    #print(D1, max(C1), min(C1))
    #print(D2, max(C2), min(C2))
    # print(D0, D1)
    if D0 > D1:
        return 0, D0/(D0+D1)
    else:
        return 1, D1/(D0+D1)

In [6]:
def Acc(A,B):
    result = 0

    for i in range(len(A)):
        if int(A[i]) - int(B[i]) == 0:
            result += 1

    return round(result*100/len(A), 2)


def Tprecision(Pre,Act):
    result = 0
    TP = 0
    FP = 0

    for i in range(len(Pre)):
        if int(Pre[i]) == 1 and int(Act[i]) == 1:
            TP +=1
        if int(Pre[i]) == 1 and int(Act[i]) == 0:
            FP +=1

    #return str(TP) +  " : " + str(TP+FP)

    if TP:
        return round(100*TP/(TP+FP),2)
    else:
        if FP:
            return 0
        else:
            return None


def Trecall(Pre,Act):
    result = 0
    TP = 0
    FN = 0

    for i in range(len(Pre)):
        if int(Pre[i]) == 1 and int(Act[i]) == 1:
            TP +=1
        if int(Pre[i]) == 0 and int(Act[i]) == 1:
            FN +=1

    #return str(TP) +  " : " + str(TP+FN)

    if TP:
        return round(100*TP/(TP+FN),2)
    else:
        if FN:
            return 0
        else:
            return None

# listAcc = []
# listPre = []
# listRe = []
# timeTest = []
# timeUpdate = []
# listRank = []
# res = []
##############################
import matplotlib.pyplot as plt

from sklearn import metrics


###################################
def testAccuracy(base,Te,C):
    test = Te
    X = np.zeros(len(test))
    ddd = np.zeros(len(test))
    X_test = np.array(test).T[-1]
    for i in range(len(test)):
        X[i], ddd[i] = FISA(base, C, test[i])
        # listRank.append(ddd[i])
        #print(test[i])
    # for dd in ddd:
    #     print(dd)
    # listRank.append(ddd)

    #print(X)
    #print(X_test)
    # res.append(X)
    # listAcc.append(Acc(X,X_test))
    # listPre.append(Tprecision(X,X_test))
    # listRe.append(Trecall(X,X_test))
    actual = np.array(X_test).astype('float64')
    predicted = np.array(X).astype('float64')
    # print(actual)
    # print(predicted)
    # print(predicted)
    # print(X_test)
    # print(X)
    # confusion_matrix = metrics.confusion_matrix(actual, predicted)

    # cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0, 1])

    # cm_display.plot()
    # plt.show()
    return Acc(X,X_test), Tprecision(X,X_test), Trecall(X,X_test)

In [7]:
def scenario_FKGS(ran, e):

    traindf = pd.read_csv('train.csv')
    testdf = pd.read_csv('test.csv')
    train = traindf.values.tolist()
    test = testdf.values.tolist()


    sampling_time = []
    train_time = []
    test_time = []
    accuracy = []
    precision = []
    recall = []
    for i in range(10):
        start = time.time()
        base = sampling(ran=ran, base=train, e=e)
        totalTime = time.time() - start
        sampling_time.append(totalTime)
        print(f'len base {i}:', len(base))


        start = time.time()
        A = caculateA(base)
        M = caculateM(base)
        B = caculateB(base,A,M)
        C = caculateC(base,B)
        totalTime = time.time() - start
        train_time.append(totalTime)


        start = time.time()
        acc,pre,rec = testAccuracy(base,test,C)
        totalTime = time.time() - start
        accuracy.append(acc)
        precision.append(pre)
        recall.append(rec)
        test_time.append(totalTime)


    print('sampling_time:', sampling_time)
    print('train_time:', train_time)
    print('test_time:', test_time)
    print('accuracy:', accuracy)
    print('precision:', precision)
    print('recall:', recall)


In [8]:
def scenario_random(ran):

    traindf = pd.read_csv('train.csv')
    testdf = pd.read_csv('test.csv')
    test = testdf.values.tolist()

    sampling_time = []
    train_time = []
    test_time = []
    accuracy = []
    precision = []
    recall = []
    for i in range(10):
        
        start = time.time()
        train = traindf.sample(n=int(len(traindf)*ran/100))
        base = train.values.tolist()
        totalTime = time.time() - start
        sampling_time.append(totalTime)
        print(f'len base {i}:', len(base))


        start = time.time()
        A = caculateA(base)
        M = caculateM(base)
        B = caculateB(base,A,M)
        C = caculateC(base,B)
        totalTime = time.time() - start
        train_time.append(totalTime)


        start = time.time()
        acc,pre,rec = testAccuracy(base,test,C)
        totalTime = time.time() - start
        accuracy.append(acc)
        precision.append(pre)
        recall.append(rec)
        test_time.append(totalTime)

    print('sampling_time:', sampling_time)
    print('train_time:', train_time)
    print('test_time:', test_time)
    print('accuracy:', accuracy)
    print('precision:', precision)
    print('recall:', recall)


In [9]:
def scenario_full():

    traindf = pd.read_csv('train.csv')
    testdf = pd.read_csv('test.csv')
    test = testdf.values.tolist()

    sampling_time = []
    train_time = []
    test_time = []
    accuracy = []
    precision = []
    recall = []
    for i in range(1):
        
        start = time.time()
        train = traindf
        base = train.values.tolist()
        totalTime = time.time() - start
        sampling_time.append(totalTime)
        print(f'len base {i}:', len(base))


        start = time.time()
        A = caculateA(base)
        M = caculateM(base)
        B = caculateB(base,A,M)
        C = caculateC(base,B)
        totalTime = time.time() - start
        train_time.append(totalTime)


        start = time.time()
        acc,pre,rec = testAccuracy(base,test,C)
        totalTime = time.time() - start
        accuracy.append(acc)
        precision.append(pre)
        recall.append(rec)
        test_time.append(totalTime)

    print('sampling_time:', sampling_time)
    print('train_time:', train_time)
    print('test_time:', test_time)
    print('accuracy:', accuracy)
    print('precision:', precision)
    print('recall:', recall)

In [10]:
scenario_FKGS(10, 0.1)

len base 0: 68
done A
done B
done C
len base 1: 65
done A
done B
done C
len base 2: 68
done A
done B
done C
len base 3: 68
done A
done B
done C
len base 4: 66
done A
done B
done C
len base 5: 68
done A
done B
done C
len base 6: 67
done A
done B
done C
len base 7: 67
done A
done B
done C
len base 8: 66
done A
done B
done C
len base 9: 65
done A
done B
done C
sampling_time: [0.0009937286376953125, 0.0010004043579101562, 0.0010023117065429688, 0.0009999275207519531, 0.0009996891021728516, 0.0, 0.0010004043579101562, 0.0009922981262207031, 0.0010013580322265625, 0.0009946823120117188]
train_time: [0.8759996891021729, 0.8199996948242188, 0.9629995822906494, 0.8790059089660645, 0.9029941558837891, 0.9959938526153564, 0.9059994220733643, 0.8580000400543213, 0.8790004253387451, 0.8339972496032715]
test_time: [0.32500505447387695, 0.3049931526184082, 0.33299946784973145, 0.31799983978271484, 0.3200061321258545, 0.3220059871673584, 0.31800150871276855, 0.3189985752105713, 0.3110063076019287, 0.3

In [11]:
scenario_FKGS(10, 0.2)

len base 0: 66
done A
done B
done C
len base 1: 66
done A
done B
done C
len base 2: 68
done A
done B
done C
len base 3: 67
done A
done B
done C
len base 4: 68
done A
done B
done C
len base 5: 68
done A
done B
done C
len base 6: 64
done A
done B
done C
len base 7: 65
done A
done B
done C
len base 8: 65
done A
done B
done C
len base 9: 67
done A
done B
done C
sampling_time: [0.0009996891021728516, 0.0009996891021728516, 0.0010006427764892578, 0.0009999275207519531, 0.0009999275207519531, 0.0010073184967041016, 0.00099945068359375, 0.0010018348693847656, 0.0, 0.001001119613647461]
train_time: [0.8390047550201416, 0.8239941596984863, 0.9200067520141602, 0.8719959259033203, 0.8530006408691406, 0.8359982967376709, 0.7990028858184814, 0.8530001640319824, 0.8139994144439697, 0.8570055961608887]
test_time: [0.3120005130767822, 0.3169982433319092, 0.32400012016296387, 0.3130042552947998, 0.31499361991882324, 0.3170008659362793, 0.298997163772583, 0.31099438667297363, 0.2999992370605469, 0.323000

In [12]:
scenario_FKGS(10, 0.3)

len base 0: 65
done A
done B
done C
len base 1: 65
done A
done B
done C
len base 2: 64
done A
done B
done C
len base 3: 65
done A
done B
done C
len base 4: 66
done A
done B
done C
len base 5: 66
done A
done B
done C
len base 6: 65
done A
done B
done C
len base 7: 64
done A
done B
done C
len base 8: 64
done A
done B
done C
len base 9: 65
done A
done B
done C
sampling_time: [0.0010013580322265625, 0.0010001659393310547, 0.001001119613647461, 0.001001596450805664, 0.0009999275207519531, 0.0009999275207519531, 0.0010001659393310547, 0.001001596450805664, 0.0010004043579101562, 0.000993490219116211]
train_time: [0.7730040550231934, 0.7880051136016846, 0.7729923725128174, 0.7849984169006348, 0.7639999389648438, 0.7849934101104736, 0.7700045108795166, 0.7369985580444336, 0.7739999294281006, 0.7600064277648926]
test_time: [0.300994873046875, 0.30500006675720215, 0.3019986152648926, 0.29900622367858887, 0.29400014877319336, 0.3079991340637207, 0.2980003356933594, 0.29399991035461426, 0.29699945

In [13]:
scenario_FKGS(10, 0.4)

len base 0: 66
done A
done B
done C
len base 1: 65
done A
done B
done C
len base 2: 64
done A
done B
done C
len base 3: 64
done A
done B
done C
len base 4: 65
done A
done B
done C
len base 5: 64
done A
done B
done C
len base 6: 66
done A
done B
done C
len base 7: 65
done A
done B
done C
len base 8: 65
done A
done B
done C
len base 9: 67
done A
done B
done C
sampling_time: [0.0009999275207519531, 0.0019996166229248047, 0.00099945068359375, 0.0009989738464355469, 0.000993490219116211, 0.0009992122650146484, 0.0010004043579101562, 0.0010008811950683594, 0.0010004043579101562, 0.0009992122650146484]
train_time: [0.7930052280426025, 0.7419998645782471, 0.7949998378753662, 0.7579379081726074, 0.7530062198638916, 0.7339992523193359, 0.7920072078704834, 0.7960624694824219, 0.7620062828063965, 0.7920000553131104]
test_time: [0.3100004196166992, 0.3040003776550293, 0.3060641288757324, 0.2929995059967041, 0.29000115394592285, 0.2979927062988281, 0.30700016021728516, 0.3089301586151123, 0.29900050

In [14]:
scenario_FKGS(10, 0.5)

len base 0: 64
done A
done B
done C
len base 1: 64
done A
done B
done C
len base 2: 64
done A
done B
done C
len base 3: 65
done A
done B
done C
len base 4: 64
done A
done B
done C
len base 5: 64
done A
done B
done C
len base 6: 64
done A
done B
done C
len base 7: 65
done A
done B
done C
len base 8: 65
done A
done B
done C
len base 9: 64
done A
done B
done C
sampling_time: [0.0019991397857666016, 0.0009999275207519531, 0.001999378204345703, 0.0010018348693847656, 0.0010013580322265625, 0.0010004043579101562, 0.0010077953338623047, 0.0010004043579101562, 0.001999378204345703, 0.0020003318786621094]
train_time: [0.8069913387298584, 0.8239996433258057, 0.7770049571990967, 0.8129978179931641, 0.7969355583190918, 0.7710011005401611, 0.7659926414489746, 0.8130009174346924, 0.777001142501831, 0.7839312553405762]
test_time: [0.3030080795288086, 0.29999613761901855, 0.2990000247955322, 0.309063196182251, 0.30399250984191895, 0.3019990921020508, 0.30499863624572754, 0.30600619316101074, 0.3010687

In [15]:
scenario_random(10)

len base 0: 63
done A
done B
done C
len base 1: 63
done A
done B
done C
len base 2: 63
done A
done B
done C
len base 3: 63
done A
done B
done C
len base 4: 63
done A
done B
done C
len base 5: 63
done A
done B
done C
len base 6: 63
done A
done B
done C
len base 7: 63
done A
done B
done C
len base 8: 63
done A
done B
done C
len base 9: 63
done A
done B
done C
sampling_time: [0.0010077953338623047, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0010001659393310547, 0.0]
train_time: [0.7409923076629639, 0.7820062637329102, 0.7919998168945312, 0.7480061054229736, 0.8119933605194092, 0.8069925308227539, 0.7599935531616211, 0.7780003547668457, 0.7910716533660889, 0.821007490158081]
test_time: [0.29399967193603516, 0.29600024223327637, 0.30199432373046875, 0.2859995365142822, 0.29700660705566406, 0.2990076541900635, 0.29500627517700195, 0.2929985523223877, 0.301922082901001, 0.299999475479126]
accuracy: [79.2, 61.31, 81.39, 71.9, 82.85, 74.09, 78.47, 62.04, 61.68, 82.48]
precision: [83.21, 87.5, 79.04, 

In [16]:
scenario_FKGS(15, 0.1)

len base 0: 97
done A
done B
done C
len base 1: 98
done A
done B
done C
len base 2: 96
done A
done B
done C
len base 3: 100
done A
done B
done C
len base 4: 97
done A
done B
done C
len base 5: 97
done A
done B
done C
len base 6: 96
done A
done B
done C
len base 7: 98
done A
done B
done C
len base 8: 97
done A
done B
done C
len base 9: 96
done A
done B
done C
sampling_time: [0.0019922256469726562, 0.0019998550415039062, 0.002001047134399414, 0.0019996166229248047, 0.001993417739868164, 0.002000093460083008, 0.0019943714141845703, 0.0019996166229248047, 0.0020003318786621094, 0.002000093460083008]
train_time: [1.481006383895874, 1.572000503540039, 1.5829997062683105, 1.6480011940002441, 1.6089990139007568, 1.5920002460479736, 1.6270065307617188, 1.729008436203003, 1.6009976863861084, 1.5089941024780273]
test_time: [0.44100069999694824, 0.4469926357269287, 0.4680049419403076, 0.4630002975463867, 0.44800758361816406, 0.44601941108703613, 0.4489917755126953, 0.45499348640441895, 0.454007387

In [17]:
scenario_FKGS(15, 0.2)

len base 0: 99
done A
done B
done C
len base 1: 99
done A
done B
done C
len base 2: 99
done A
done B
done C
len base 3: 99
done A
done B
done C
len base 4: 99
done A
done B
done C
len base 5: 98
done A
done B
done C
len base 6: 99
done A
done B
done C
len base 7: 96
done A
done B
done C
len base 8: 99
done A
done B
done C
len base 9: 99
done A
done B
done C
sampling_time: [0.0019903182983398438, 0.001992940902709961, 0.001999378204345703, 0.0019998550415039062, 0.0019998550415039062, 0.0019998550415039062, 0.002000093460083008, 0.002000093460083008, 0.0020062923431396484, 0.0020003318786621094]
train_time: [1.5789995193481445, 1.7179956436157227, 1.6680057048797607, 1.7039997577667236, 1.6449999809265137, 1.566999912261963, 1.5669996738433838, 1.4970076084136963, 1.589994192123413, 1.5440003871917725]
test_time: [0.45500779151916504, 0.46799540519714355, 0.4629998207092285, 0.4760005474090576, 0.45399999618530273, 0.44899988174438477, 0.44799232482910156, 0.437000036239624, 0.456999540

In [18]:
scenario_FKGS(15, 0.3)

len base 0: 96
done A
done B
done C
len base 1: 99
done A
done B
done C
len base 2: 96
done A
done B
done C
len base 3: 97
done A
done B
done C
len base 4: 96
done A
done B
done C
len base 5: 99
done A
done B
done C
len base 6: 96
done A
done B
done C
len base 7: 98
done A
done B
done C
len base 8: 98
done A
done B
done C
len base 9: 99
done A
done B
done C
sampling_time: [0.0019996166229248047, 0.002000093460083008, 0.0019996166229248047, 0.002000093460083008, 0.003000020980834961, 0.002000093460083008, 0.002006053924560547, 0.00200653076171875, 0.002000093460083008, 0.0019998550415039062]
train_time: [1.5130071640014648, 1.5739943981170654, 1.5059938430786133, 1.4839918613433838, 1.475006341934204, 1.6040012836456299, 1.4990038871765137, 1.4729995727539062, 1.5920023918151855, 1.5270001888275146]
test_time: [0.4420003890991211, 0.45500612258911133, 0.4400064945220947, 0.4400017261505127, 0.4419991970062256, 0.45699310302734375, 0.44998979568481445, 0.4479999542236328, 0.4559977054595

In [19]:
scenario_FKGS(15, 0.4)

len base 0: 97
done A
done B
done C
len base 1: 96
done A
done B
done C
len base 2: 97
done A
done B
done C
len base 3: 96
done A
done B
done C
len base 4: 98
done A
done B
done C
len base 5: 96
done A
done B
done C
len base 6: 98
done A
done B
done C
len base 7: 96
done A
done B
done C
len base 8: 96
done A
done B
done C
len base 9: 97
done A
done B
done C
sampling_time: [0.0030002593994140625, 0.002000093460083008, 0.0029985904693603516, 0.002999544143676758, 0.0020003318786621094, 0.0019996166229248047, 0.001992464065551758, 0.003000020980834961, 0.0019998550415039062, 0.0019998550415039062]
train_time: [1.4709994792938232, 1.5019922256469727, 1.5489943027496338, 1.4949939250946045, 1.560999870300293, 1.4419946670532227, 1.4880058765411377, 1.397994041442871, 1.464000940322876, 1.4740002155303955]
test_time: [0.44699978828430176, 0.4410090446472168, 0.45500612258911133, 0.4450058937072754, 0.45400047302246094, 0.4470064640045166, 0.4370002746582031, 0.43399882316589355, 0.4620063304

In [20]:
scenario_FKGS(15, 0.5)

len base 0: 96
done A
done B
done C
len base 1: 96
done A
done B
done C
len base 2: 96
done A
done B
done C
len base 3: 97
done A
done B
done C
len base 4: 97
done A
done B
done C
len base 5: 97
done A
done B
done C
len base 6: 97
done A
done B
done C
len base 7: 96
done A
done B
done C
len base 8: 97
done A
done B
done C
len base 9: 97
done A
done B
done C
sampling_time: [0.003000020980834961, 0.0030002593994140625, 0.0029997825622558594, 0.0029997825622558594, 0.003999948501586914, 0.004003047943115234, 0.003000497817993164, 0.00200653076171875, 0.0029985904693603516, 0.002998828887939453]
train_time: [1.5530059337615967, 1.5099997520446777, 1.5360066890716553, 1.6479978561401367, 1.7129936218261719, 1.5670013427734375, 1.7149925231933594, 1.526994228363037, 1.5520000457763672, 1.5230000019073486]
test_time: [0.4440014362335205, 0.44399237632751465, 0.44199538230895996, 0.4670085906982422, 0.4790005683898926, 0.4500007629394531, 0.4720005989074707, 0.46500658988952637, 0.455001592636

In [21]:
scenario_random(15)

len base 0: 95
done A
done B
done C
len base 1: 95
done A
done B
done C
len base 2: 95
done A
done B
done C
len base 3: 95
done A
done B
done C
len base 4: 95
done A
done B
done C
len base 5: 95
done A
done B
done C
len base 6: 95
done A
done B
done C
len base 7: 95
done A
done B
done C
len base 8: 95
done A
done B
done C
len base 9: 95
done A
done B
done C
sampling_time: [0.0, 0.0010023117065429688, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0009999275207519531, 0.0, 0.0010058879852294922]
train_time: [1.5637757778167725, 1.4759979248046875, 1.5969996452331543, 1.5100667476654053, 1.6199924945831299, 1.584993600845337, 1.5650057792663574, 1.6070001125335693, 1.5500080585479736, 1.57200026512146]
test_time: [0.4499940872192383, 0.4370002746582031, 0.45900583267211914, 0.43993330001831055, 0.46600794792175293, 0.44900035858154297, 0.4500000476837158, 0.4469931125640869, 0.43499279022216797, 0.44700002670288086]
accuracy: [66.06, 79.2, 79.56, 76.64, 78.1, 77.37, 83.21, 56.93, 70.07, 82.12]
precision: [6

In [22]:
scenario_FKGS(20, 0.1)

len base 0: 130
done A
done B
done C
len base 1: 128
done A
done B
done C
len base 2: 129
done A
done B
done C
len base 3: 130
done A
done B
done C
len base 4: 130
done A
done B
done C
len base 5: 129
done A
done B
done C
len base 6: 130
done A
done B
done C
len base 7: 130
done A
done B
done C
len base 8: 132
done A
done B
done C
len base 9: 129
done A
done B
done C
sampling_time: [0.003992319107055664, 0.003000020980834961, 0.0030057430267333984, 0.0029997825622558594, 0.002999544143676758, 0.0030066967010498047, 0.0039942264556884766, 0.0029931068420410156, 0.003999471664428711, 0.003994941711425781]
train_time: [2.7130062580108643, 2.686000108718872, 2.74299955368042, 2.6739933490753174, 2.658010959625244, 2.7269999980926514, 2.6950058937072754, 2.7340006828308105, 2.796006202697754, 2.800997257232666]
test_time: [0.6069998741149902, 0.5899946689605713, 0.6090004444122314, 0.6319997310638428, 0.6069891452789307, 0.6029996871948242, 0.6160001754760742, 0.6129932403564453, 0.63200020

In [23]:
scenario_FKGS(20, 0.2)

len base 0: 128
done A
done B
done C
len base 1: 130
done A
done B
done C
len base 2: 130
done A
done B
done C
len base 3: 128
done A
done B
done C
len base 4: 130
done A
done B
done C
len base 5: 130
done A
done B
done C
len base 6: 130
done A
done B
done C
len base 7: 129
done A
done B
done C
len base 8: 130
done A
done B
done C
len base 9: 128
done A
done B
done C
sampling_time: [0.0029997825622558594, 0.0030019283294677734, 0.003998279571533203, 0.004001617431640625, 0.0040781497955322266, 0.004999876022338867, 0.002999544143676758, 0.004006385803222656, 0.003999948501586914, 0.003992795944213867]
train_time: [2.756002902984619, 2.6369986534118652, 2.682933807373047, 2.8239269256591797, 2.849928617477417, 2.8570010662078857, 2.632004499435425, 2.613003730773926, 2.7599997520446777, 2.7090365886688232]
test_time: [0.6120047569274902, 0.6120672225952148, 0.6340668201446533, 0.6499972343444824, 0.6240010261535645, 0.6379947662353516, 0.6129934787750244, 0.6359965801239014, 0.600000143

In [24]:
scenario_FKGS(20, 0.3)

len base 0: 128
done A
done B
done C
len base 1: 129
done A
done B
done C
len base 2: 128
done A
done B
done C
len base 3: 132
done A
done B
done C
len base 4: 129
done A
done B
done C
len base 5: 130
done A
done B
done C
len base 6: 128
done A
done B
done C
len base 7: 129
done A
done B
done C
len base 8: 131
done A
done B
done C
len base 9: 129
done A
done B
done C
sampling_time: [0.0040073394775390625, 0.0039997100830078125, 0.003999233245849609, 0.003993511199951172, 0.003999948501586914, 0.004000186920166016, 0.004000663757324219, 0.0040035247802734375, 0.003991127014160156, 0.0029997825622558594]
train_time: [2.4690005779266357, 2.5260000228881836, 2.4049935340881348, 2.516000986099243, 2.448007583618164, 2.6259939670562744, 2.442000389099121, 2.577002763748169, 2.5839996337890625, 2.490000009536743]
test_time: [0.5999939441680908, 0.5910062789916992, 0.5860064029693604, 0.6069982051849365, 0.5859999656677246, 0.6020050048828125, 0.5949959754943848, 0.6020064353942871, 0.60000014

In [25]:
scenario_FKGS(20, 0.4)

len base 0: 129
done A
done B
done C
len base 1: 128
done A
done B
done C
len base 2: 129
done A
done B
done C
len base 3: 129
done A
done B
done C
len base 4: 128
done A
done B
done C
len base 5: 129
done A
done B
done C
len base 6: 128
done A
done B
done C
len base 7: 129
done A
done B
done C
len base 8: 129
done A
done B
done C
len base 9: 128
done A
done B
done C
sampling_time: [0.003997325897216797, 0.003998517990112305, 0.0039997100830078125, 0.003999948501586914, 0.004999876022338867, 0.0039997100830078125, 0.0039997100830078125, 0.004999637603759766, 0.003999948501586914, 0.0039937496185302734]
train_time: [2.4940009117126465, 2.5250000953674316, 2.5229997634887695, 2.4879987239837646, 2.460007905960083, 2.450000047683716, 2.5419998168945312, 2.4880001544952393, 2.59501051902771, 2.446000337600708]
test_time: [0.5820081233978271, 0.581000566482544, 0.5920000076293945, 0.6089935302734375, 0.5800001621246338, 0.5800004005432129, 0.5860002040863037, 0.5829925537109375, 0.610997200

In [26]:
scenario_FKGS(20, 0.5)

len base 0: 129
done A
done B
done C
len base 1: 128
done A
done B
done C
len base 2: 129
done A
done B
done C
len base 3: 129
done A
done B
done C
len base 4: 128
done A
done B
done C
len base 5: 128
done A
done B
done C
len base 6: 128
done A
done B
done C
len base 7: 128
done A
done B
done C
len base 8: 128
done A
done B
done C
len base 9: 128
done A
done B
done C
sampling_time: [0.003999471664428711, 0.004999876022338867, 0.0050046443939208984, 0.004999637603759766, 0.004997730255126953, 0.00599217414855957, 0.005006551742553711, 0.004999399185180664, 0.003998756408691406, 0.005001544952392578]
train_time: [2.522008180618286, 2.5109996795654297, 2.578993558883667, 2.562999963760376, 2.5490012168884277, 2.6120071411132812, 2.5369999408721924, 2.6920006275177, 2.553004026412964, 2.557006597518921]
test_time: [0.5919992923736572, 0.5759952068328857, 0.6010069847106934, 0.5910012722015381, 0.622999906539917, 0.5949938297271729, 0.5930006504058838, 0.5980002880096436, 0.5979888439178467

In [27]:
scenario_random(20)

len base 0: 127
done A
done B
done C
len base 1: 127
done A
done B
done C
len base 2: 127
done A
done B
done C
len base 3: 127
done A
done B
done C
len base 4: 127
done A
done B
done C
len base 5: 127
done A
done B
done C
len base 6: 127
done A
done B
done C
len base 7: 127
done A
done B
done C
len base 8: 127
done A
done B
done C
len base 9: 127
done A
done B
done C
sampling_time: [0.0, 0.0010001659393310547, 0.0, 0.0010027885437011719, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0009937286376953125]
train_time: [2.5040056705474854, 2.682999610900879, 2.54400897026062, 2.506002902984619, 2.736990213394165, 2.5940020084381104, 2.5259950160980225, 2.568006992340088, 2.716278314590454, 2.6840007305145264]
test_time: [0.5820000171661377, 0.5909998416900635, 0.5829861164093018, 0.5840015411376953, 0.5930001735687256, 0.588005542755127, 0.5839989185333252, 0.5759997367858887, 0.6210057735443115, 0.6170058250427246]
accuracy: [80.29, 62.77, 62.41, 66.79, 71.53, 79.56, 83.58, 62.41, 63.5, 78.1]
precision: [78

In [28]:
scenario_full()

len base 0: 638
done A
done B
done C
sampling_time: [0.0]
train_time: [55.049004316329956]
test_time: [3.108001708984375]
accuracy: [79.56]
precision: [84.85]
recall: [75.68]
